<h1> Price Prediction for Cars </h1>
using given sample data

* Step 1: testing if python setup for jupiter notebook is working

In [ ]:
print("hello world")

* Step 2: install the libraries

In [ ]:
! pip install boto3
! pip install pandas
! pip install scikit-learn

* Step 3: visualize the table

In [2]:
import pandas as pd

df = pd.read_csv('data/sample_data.csv')
df

,car_ID,CarName,ownername,owneremail,dealershipaddress,saledate,iban,fueltype,aspiration,doornumber,...,carheight,curbweight,cylindernumber,enginesize,compressionratio,horsepower,peakrpm,citympg,highwaympg,Price
0,1,toyota corolla,Matthew Hernandez,matthew.hernandez@gmail.com,"96559 Moore Street\r\nPenningtonchester, NJ 99785",2016-11-23,GB84TJXJ3114**********,diesel,std,four,...,52.8,2275,4.0,110,22.5,56.0,4500,38,47,7788.0
1,2,toyota carina,Megan Sanchez,megan.sanchez@gmail.com,18399 House Village Apt. 697\r\nLake Amberburg...,2020-08-11,GB13IBBH0818**********,gas,std,four,...,59.1,3110,4.0,92,9.0,62.0,4800,27,32,8778.0
2,3,toyota corolla,Sarah Lawrence,sarahlawrence@gmail.com,"79775 Anne Way\r\nEast Ashley, GU 04171",2018-06-26,GB07IVPZ1581**********,gas,std,two,...,52.0,2714,4.0,146,9.3,116.0,4800,24,30,11549.0
3,4,toyota corona,Debra Juarez,debra.juarez@hotmail.com,"540 Allen Lane\r\nLake Randy, ME 90623",2018-07-24,GB69IASI9434**********,gas,std,two,...,52.0,3016,6.0,171,9.3,161.0,5200,19,24,15998.0
4,5,nissan otti,Michael Harvey,michaelharvey@gmail.com,5519 Krause Station Apt. 935\r\nWest Frederick...,2022-02-10,GB33TRRF7833**********,gas,std,four,...,55.1,3060,6.0,181,9.0,152.0,5200,19,25,13499.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,241,volkswagen type 3,Kelly Cook,kellycook@yahoo.com,"10091 Tiffany Expressway\r\nNorth Nathaniel, S...",2018-09-06,GB96KBQO3167**********,gas,std,four,...,55.7,2212,4.0,109,9.0,85.0,5250,27,34,8195.0
241,242,NaN,Hannah Davis,hannah.davis@gmail.com,"013 Allen Spurs Apt. 359\r\nEast Veronica, PR ...",2019-10-07,GB29KTVT0789**********,NaN,turbo,two,...,49.4,2370,4.0,110,7.5,NaN,5500,23,30,9978.0
242,243,bmw z4,Sarah Smith,sarahsmith@yahoo.com,"48889 Deleon Square\r\nSouth Aaron, UT 20573",2021-09-23,GB70DUZB1172**********,gas,std,four,...,55.7,3055,6.0,164,9.0,121.0,4250,20,25,24565.0
243,244,nissan kicks,Rhonda Grant,rhondagrant@gmail.com,"104 Valdez Neck\r\nJohnsonmouth, AK 90570",2015-01-22,GB12LNDW6015**********,gas,turbo,two,...,49.7,3139,6.0,181,7.8,200.0,5200,17,23,19699.0


* Step 4: Remove Duplicate records

in our sample there are no duplicate records. however keeping the cleanup step will help future datasets

In [3]:
# check how many duplicates exist
duplicates = df[df.duplicated(keep=False)]
print(duplicates.to_string(index=False))

df = df.drop_duplicates()

Empty DataFrame
Columns: [car_ID, CarName, ownername, owneremail, dealershipaddress, saledate, iban, fueltype, aspiration, doornumber, carbody, drivewheel, enginelocation, wheelbase, color, carlength, carwidth, carheight, curbweight, cylindernumber, enginesize, compressionratio, horsepower, peakrpm, citympg, highwaympg, Price]
Index: []


* Step 5: eleminate records with NaN/Null values

In general, if the NaN's are derivable, they should be derived. 

Note: But beaware that unknown value is not zero. that may express better results but that would corrupt the model to unrealisitic training.

In [4]:
selected_columns = ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'carheight', 'curbweight', 'carlength', 'cylindernumber', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'Price']

df = df.drop_duplicates(selected_columns, keep=False)
df

null_columns = df.columns[df.isnull().any()].tolist()
null_columns

columns_check = ['CarName', 'fueltype',
 'carbody',
 'enginelocation',
 'carlength',
 'cylindernumber',
 'horsepower']

rows_with_nulls = df[df[columns_check].isnull().any(axis=1)]
rows_with_nulls

df = df.dropna(subset=selected_columns)
df

rows_with_nulls = df[df[columns_check].isnull().any(axis=1)]
rows_with_nulls[columns_check]

,CarName,fueltype,carbody,enginelocation,carlength,cylindernumber,horsepower


* Step 6: ensure their are no corruption in data and if they are try to fix them or eleminate them from the sample.
* Step 7: remove the columns that are unwanted.

In [5]:
# Cleaning data
selected_columns = ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'carheight', 'curbweight', 'cylindernumber', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'Price']
# df['carsize'] = df['carheight'] * df['curbweight'] * df['enginesize']

df = df.drop_duplicates(selected_columns)
tiny_df = df[selected_columns]
filtered_df = tiny_df[tiny_df[selected_columns].notna()]

# filtered_df.head()
sorted_df = filtered_df.sort_values(by='Price', ascending=True)
#print(sorted_df);
sorted_df.head()


,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,carheight,curbweight,cylindernumber,enginesize,compressionratio,horsepower,peakrpm,citympg,highwaympg,Price
48,subaru,gas,std,two,hatchback,fwd,front,53.7,2050,4.0,97,9.0,69.0,4900,31,36,5118.0
244,chevrolet impala,gas,std,two,hatchback,fwd,front,53.2,1488,3.0,61,9.5,48.0,5100,47,53,5151.0
148,maxda rx3,gas,std,two,hatchback,fwd,front,54.1,1890,4.0,91,9.0,68.0,5000,30,31,5195.0
30,toyota corona mark ii,gas,std,two,hatchback,fwd,front,54.5,1985,4.0,92,9.0,62.0,4800,35,39,5348.0
174,mitsubishi mirage,gas,std,two,hatchback,fwd,front,50.8,1918,4.0,92,9.4,68.0,5500,37,41,5389.0


<h2> Training the model </h2>
* This is typical problem of prediction of a numerical value based on given parameter.
* The model is basically a curve fitting problem where x are given properties and y is a point we are interested interested in.

Regression Linear/Curve[non linear]

NOTE: Our dataset is very small therefore Neural Networks wont be good approach. 
1.  define features, target, identify categorical feature and numeric features.
2.  build the pipelines
3.  split the dataset into Training and Testing dataset. 
4.  execute training process (fit)
5.  execute the Test and evaluate the performance. (predict for the known targets)
6.  execute the Predict on real world data to predict the Price

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

features = ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'carheight', 'curbweight', 'cylindernumber', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']
target = 'Price'

x = df[features]
y = df[target]

categorical_features = ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation']
numeric_features = ['carheight', 'curbweight', 'cylindernumber', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']

# Imputers for column transformation
categorical_imputer = SimpleImputer(strategy='most_frequent')
numeric_imputer = SimpleImputer(strategy='mean')
preprocessor = ColumnTransformer(transformers=[
    ('cat', Pipeline([
        ('imputer', categorical_imputer),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_features),
    ('num', numeric_imputer, numeric_features)
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # ('regressor', LinearRegression())
    ('RandomForest', RandomForestRegressor(random_state=42))
])

<h2> Training the model </h2>

In [7]:
# Train-test split and train
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
pipeline.fit(x_train, y_train)

,steps,"[('preprocessor', ...), ('RandomForest', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


<h2> Performance Evaluation </h2>
The model is not performing very well. the Mean Squared Error is significantly high. 
Mean Squared Error: 7147802.95 is significanly high value.
apparently the dataset is of large range too


In [ ]:
# Predict and evaluate
y_pred = pipeline.predict(x_test)
mse = mean_squared_error(y_test, y_pred)

# Output results
print("Predicted Prices:", y_pred)
print("Mean Squared Error:", mse)

# Error analysis
diff  = y_test - y_pred
print(f" mean = {diff.mean()} \n max= {diff.max()} \n min = {diff.min()} \n standard deviation = {diff.std()}")


Predicted Prices: [ 7569.21416667 19117.39883333  7290.22833333 16405.08002
 16635.2325      9383.75166667 14374.6125     11677.40900333
 29844.35        9947.129      16602.41417     7264.74
  6646.28666667 38641.325      14311.62        5928.62
  9361.485       5828.11        7826.58583333 14304.625
  7385.94916667  8846.78066667]
Mean Squared Error: 7147802.95166467
 mean = 330.7978396969698 
 max= 5447.601166666664 
 min = -7881.324999999997 
 standard deviation = 2715.4256554882772
